In [1]:
from collections import defaultdict
import numpy as np
import time

import ray

print('Successfully imported ray!')

Successfully imported ray!


# 1. Introducing Actors
Classes decorated with `ray.remote()` turn it into a Remote Actor

In [2]:
ray.init(num_cpus=4, ignore_reinit_error=True)

2019-12-20 16:54:24,812	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2019-12-20 16:54:24,816	INFO resource_spec.py:216 -- Starting Ray with 1.71 GiB memory available for workers and up to 0.86 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '10.2.216.143',
 'redis_address': '10.2.216.143:61257',
 'object_store_address': '/tmp/ray/session_2019-12-20_16-54-24_810507_5473/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-12-20_16-54-24_810507_5473/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-12-20_16-54-24_810507_5473'}

In [3]:
@ray.remote
class Foo(object):
    def __init__(self):
        self.counter = 0

    def reset(self):
        self.counter = 0

    def increment(self):
        time.sleep(0.5)
        self.counter += 1
        return self.counter

assert hasattr(Foo, 'remote'), 'You need to turn "Foo" into an actor with @ray.remote.'

In [4]:
# Create two Foo actors.
f1 = Foo.remote()
f2 = Foo.remote()

In [7]:
start_time = time.time()

# Reset the actor state so that we can run this cell multiple times without
# changing the results.
f1.reset.remote()
f2.reset.remote()

# We want to parallelize this code. However, it is not straightforward to
# make "increment" a remote function, because state is shared (the value of
# "self.counter") between subsequent calls to "increment". In this case, it
# makes sense to use actors.
results = []
for _ in range(5):
    results.append(f1.increment.remote())
    results.append(f2.increment.remote())

results = ray.get(results)
duration = time.time() - start_time
assert not any([isinstance(result, ray.ObjectID) for result in results]), 'Looks like "results" is {}. You may have forgotten to call ray.get.'.format(results)

In [8]:
assert results == [1, 1, 2, 2, 3, 3, 4, 4, 5, 5]

assert duration < 3, ('The experiments ran in {:.3f} seconds. This is too '
                      'slow.'.format(duration))
assert duration > 2.5, ('The experiments ran in {:.3f} seconds. This is too '
                        'fast.'.format(duration))

print('Success! The example took {:.3f} seconds.'.format(duration))

Success! The example took 2.520 seconds.


# 2. Sharing References to an Actor

In [9]:
@ray.remote
class LoggingActor(object):
    def __init__(self):
        self.logs = defaultdict(lambda: [])
    
    def log(self, index, message):
        self.logs[index].append(message)
    
    def get_logs(self):
        return dict(self.logs)


assert hasattr(LoggingActor, 'remote'), ('You need to turn LoggingActor into an '
                                         'actor (by using the ray.remote keyword).')

In [10]:
logging_actor = LoggingActor.remote()

In [11]:
@ray.remote
def run_experiment(experiment_index, logging_actor):
    for i in range(60):
        time.sleep(1)
        # Push a logging message to the actor.
        logging_actor.log.remote(experiment_index, 'On iteration {}'.format(i))

In [12]:
experiment_ids = []
for i in range(3):
    experiment_ids.append(run_experiment.remote(i, logging_actor))

In [14]:
logs = ray.get(logging_actor.get_logs.remote())

assert isinstance(logs, dict), ("Make sure that you dispatch tasks to the "
                                "actor using the .remote keyword and get the results using ray.get.")
logs

{0: ['On iteration 0',
  'On iteration 1',
  'On iteration 2',
  'On iteration 3',
  'On iteration 4',
  'On iteration 5',
  'On iteration 6',
  'On iteration 7'],
 1: ['On iteration 0',
  'On iteration 1',
  'On iteration 2',
  'On iteration 3',
  'On iteration 4',
  'On iteration 5',
  'On iteration 6',
  'On iteration 7'],
 2: ['On iteration 0',
  'On iteration 1',
  'On iteration 2',
  'On iteration 3',
  'On iteration 4',
  'On iteration 5',
  'On iteration 6',
  'On iteration 7']}